In [19]:
from selenium import webdriver
from time import sleep
from webdriver_manager.chrome import ChromeDriverManager
import csv
import requests
import pandas as pd
from bs4 import BeautifulSoup


# driver = webdriver.Chrome(ChromeDriverManager().install())


url = "https://www.hltv.org/stats"
players = []
teams = []
teamRanks = []

fields1 = ['Player', 'Team', 'Maps', 'K-D_Diff', 'K/D', 'Rating(1.0)'] 
fields2 = ['Teams' ,'Maps', 'Won-Lost', 'Pistol_Win_%', 'Round_2_Conversion', 'Round_2_Break'] 
fields3 = ['Team', 'Maps', 'K-D_Diff', 'K/D', 'Rating(2.0)']

    
def writeToCSV(file, header, data):
    filename = file

    # writing to csv file 
    with open(filename, 'w') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(header) 

        # writing the data rows 
        csvwriter.writerows(data)
        
        
def get_parsed_page(url):
    # This fixes a blocked by cloudflare error i've encountered
    headers = {
        "referer": "https://www.hltv.org/stats",
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    return BeautifulSoup(requests.get(url, headers=headers).text, "lxml")



def driving():
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get('https://www.hltv.org/events/5603/blast-premier-spring-showdown-2021')
    element = driver.find_element_by_class_name('team-name')
    print(element)    
        
def playerPerformanceByDate():
    startDate = input("Start date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    endDate = input("End date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    print("Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.")
    mapName = input("Choose the map: ")
    
    dateConversion = url + "/players?startDate=" + startDate + "&endDate=" + endDate + "&maps=de_"+mapName.lower() + "&rankingFilter=Top30";
    print(dateConversion)
    soup = get_parsed_page(dateConversion)
    statTable = soup.tbody;
    for tr in statTable.find_all("tr"):
        newArray = []
        for td in tr.find_all('td'):
            newArray.append(td.get_text())
        players.append(newArray)
    writeToCSV("data.csv", fields1, players)
   

def pistolRoundByDate():
    startDate = input("Start date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    endDate = input("End date (yyyy-mm-dd, leave blank if you want 'All-time' results): ")
    print("Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.")
    mapName = input("Choose the map: ")
    teamSide = input("Enter team ('TERRORIST or COUNTER_TERRORIST'):")
    pistol = url + "/teams/pistols?startDate=" + startDate + "&endDate=" + endDate + "&maps=de_"+mapName.lower()+"&side=" + teamSide.upper() + "&rankingFilter=Top30"; 
    print(pistol)
    soup = get_parsed_page(pistol)
    pistolTable = soup.tbody;

    for tr in pistolTable.find_all("tr"):
        newArray = []
        for td in tr.find_all('td'):
            newArray.append(td.get_text())
        teams.append(newArray)
    writeToCSV("data2.csv", fields2, teams)

def teamRank():
    startDate = input("Start date (yyyy-mm-dd, leave blank for 'All-time' results): ")
    endDate = input("End date (yyyy-mm-dd, leave blank if you want 'All-time' results): ")
    print("Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.")
    mapName = input("Choose the map: ")
    teamURL = url + "/teams?startDate="+startDate+"&endDate=" + "&maps=de_"+ mapName.lower() + "&rankingFilter=Top30"; 
    print(teamURL)
    soup = get_parsed_page(teamURL)
    teamRankTable = soup.tbody;
    
    for tr in teamRankTable.find_all("tr"):
        newArray = []
        for td in tr.find_all("td"):
            newArray.append(td.get_text())
        teamRanks.append(newArray)
    writeToCSV("data3.csv", fields3, teamRanks)
    
    
    
# Calling functions and creating our csv files dynamically    
playerPerformanceByDate()
pistolRoundByDate()
teamRank()
print('-----------------------------------------------------------------------------------------------------------')
print('RUN THE 3 CELLS BELOW TO SEE THE "PLAYER DATAFRAME" AND "TEAM PISTOL DATAFRAME" AND "TEAM RANKS DATAFRAME"')
print('-----------------------------------------------------------------------------------------------------------')
    

Start date (yyyy-mm-dd, leave blank for 'All-time' results): 2021-03-01
End date (yyyy-mm-dd, leave blank for 'All-time' results): 2021-04-15
Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.
Choose the map: nuke
https://www.hltv.org/stats/players?startDate=2021-03-01&endDate=2021-04-15&maps=de_nuke&rankingFilter=Top30
Start date (yyyy-mm-dd, leave blank for 'All-time' results): 2021-03-01
End date (yyyy-mm-dd, leave blank if you want 'All-time' results): 2021-04-15
Map Selection. Choose one from the following: all, vertigo, dust2, mirage, overpass, nuke, inferno, cobblestone, cache, train.
Choose the map: nuke
Enter team ('TERRORIST or COUNTER_TERRORIST'):terrorist
https://www.hltv.org/stats/teams/pistols?startDate=2021-03-01&endDate=2021-04-15&maps=de_nuke&side=TERRORIST&rankingFilter=Top30
Start date (yyyy-mm-dd, leave blank for 'All-time' results): 2021-03-01
End date (yyyy-mm-dd, leave blank if you want '

In [20]:
#Player Performance dataframe
print("-------------------------------")
print('Player Dataframe')
print("-------------------------------")
player_df = pd.read_csv('data.csv')
player_df = player_df.dropna(axis='columns', how ='all')
player_df.head(10)


-------------------------------
Player Dataframe
-------------------------------


,Player,Maps,K-D_Diff,K/D,Rating(1.0)
0,Aerial,11,102,1.53,1.40
1,f0rest,9,45,1.31,1.29
2,El1an,9,67,1.48,1.29
3,Lack1,9,47,1.28,1.23
4,NEOFRAG,6,21,1.22,1.17
5,MUTiRiS,8,21,1.16,1.16
6,stadodo,8,14,1.10,1.16
7,oskar,6,23,1.24,1.15
8,JUST,8,22,1.16,1.14
9,ZOREE,11,40,1.22,1.13


In [21]:
# Team Pistol dataframe
print("-------------------------------")
print('Team Pistol Dataframe')
print("-------------------------------")
team_pistol_df = pd.read_csv('data2.csv')
team_pistol_df.head(10)

-------------------------------
Team Pistol Dataframe
-------------------------------


,Teams,Maps,Won-Lost,Pistol_Win_%,Round_2_Conversion,Round_2_Break
0,Heroic,3,2 - 1,66.7%,100.0%,100.0%
1,Complexity,3,2 - 1,66.7%,50.0%,100.0%
2,ex-Winstrike,6,1 - 5,16.7%,100.0%,40.0%
3,HAVU,11,3 - 8,27.3%,100.0%,12.5%
4,NIP,4,2 - 2,50.0%,50.0%,0.0%
5,Dignitas,9,7 - 2,77.8%,71.4%,0.0%
6,Winstrike,3,1 - 2,33.3%,100.0%,0.0%
7,SAW,8,2 - 6,25.0%,100.0%,0.0%
8,Sinners,6,1 - 5,16.7%,100.0%,0.0%
9,AGO,4,2 - 2,50.0%,50.0%,0.0%


In [22]:
# Team Ranks dataframe
print("-------------------------------")
print('Team Overall Rank Dataframe')
print("-------------------------------")
team_rank_df = pd.read_csv('data3.csv')
team_rank_df.head(10)

-------------------------------
Team Overall Rank Dataframe
-------------------------------


,Team,Maps,K-D_Diff,K/D,Rating(2.0)
0,Astralis,148,1458,1.12,1.09
1,Evil Geniuses,56,-11,1.00,1.07
2,North,93,105,1.01,1.07
3,Heroic,155,638,1.05,1.06
4,Liquid,112,9,1.00,1.06
5,mousesports,126,389,1.04,1.06
6,G2,127,300,1.03,1.06
7,Vitality,92,89,1.01,1.05
8,Natus Vincere,126,260,1.02,1.05
9,forZe,65,50,1.01,1.05


In [23]:




# url = 'https://www.hltv.org/events/5553/esl-pro-league-season-13'
# def eventDetails():
#     soup = get_parsed_page(url)
#     groups = soup.find_all("div", {"class": "group"})
#     for group in groups:
#         for tr in group.find_all("tr"):
#             print(tr.text)
# eventDetails()